In [ ]:
import os

%pwd

# we need to jump back to the root folder 

In [ ]:
os.chdir('../')

%pwd

In [ ]:
#3. Update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path